# Recommending movies: retrieval

Real world Recommender systems are often made up of following steps - 

* `Retrieval` :   Selecting few thousands of possible candidates to recommend from a set of millions. Because the retrieval model may be dealing with millions of candidates, it has to be computationally efficient.

* `Ranking` : Drill down thousand possible candidates to few hundreds

* `Post-Ranking` : Further refine the candidates to few dozens - Might be helpful in case user is logged from mobile device where screen real estate is limited

In this notebook, we will go over Retrieval stage of the Recommender System. Retrieval models are often composed of two sub-models:

* A `query model` computing the query representation (normally a fixed-dimensionality embedding vector) using query features.
* A `candidate model` computing the candidate representation (an equally-sized vector) using the candidate features

The outputs of the two models are then multiplied together to give a query-candidate affinity score, with higher scores expressing a better match between the candidate and the query.

## Dataset

The Movielens dataset is a classic dataset from the GroupLens research group at the University of Minnesota. It contains a set of ratings given to movies by a set of users, and is a workhorse of recommender system research.

The data can be treated in two ways:

1. It can be interpreted as expressesing which movies the users watched (and rated), and which they did not. This is a form of `implicit feedback`, where users' watches tell us which things they prefer to see and which they'd rather not see.

2. It can also be seen as expressesing how much the users liked the movies they did watch. This is a form of `explicit feedback`: given that a user watched a movie, we can tell roughly how much they liked by looking at the rating they have given.


In this tutorial, we are focusing on a retrieval system: a model that predicts a set of movies from the catalogue that the user is likely to watch. Often, implicit data is more useful here, and so we are going to treat Movielens as an implicit system. This means that every movie a user watched is a positive example, and every movie they have not seen is an implicit negative example.

## Imports

In [3]:
! pip install -q tensorflow-recommenders
! pip install -q --upgrade tensorflow-datasets
! pip install -q scann

     |████████████████████████████████| 85 kB 2.5 MB/s 
     |████████████████████████████████| 462 kB 33.2 MB/s 
     |████████████████████████████████| 4.2 MB 5.4 MB/s 
     |████████████████████████████████| 10.9 MB 4.6 MB/s 


In [4]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs